In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리 및 데이터 세트 로딩
import numpy as np
import pandas as pd
import gc #--> 구동에 필요하지 않은 메모리를 관리
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [ ]:
#데이터 불러오기

app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
#bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
#bureau_bal = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
#crdit = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv')
#instal = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv')
#pcbalance = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv')

In [ ]:
#데이터크기확인

print(app_train.shape)
print(app_test.shape)
print(prev.shape)
#print(bureau.shape)
#print(bureau_bal.shape)

In [ ]:
#데이터 전체 결합 시도하였다가 train, test, previous만 활용하는 것으로 작업 변경하였음.
#데이터 축소

prev_fe = prev[0:50000]
#bureau_fe = bureau[0:50000]
#bureau_bal_fe = bureau_bal[0:50000]
#print(prev_fe.shape)
#print(bureau_fe.shape)
#print(bureau_bal_fe.shape)

In [ ]:
#데이터 전체 결합 시도하였다가 train, test, previous만 활용하는 것으로 작업 변경하였음.
#app_trin, prev_fe, bureau_fe 데이터 결합
#app1 = app_train.merge(prev_fe, on='SK_ID_CURR', how='left')
#app1 = app1.merge(bureau_fe, on='SK_ID_CURR', how='left')
#app1.shape

In [ ]:
#데이터 전체 결합 시도하였다가 train, test, previous만 활용하는 것으로 작업 변경하였음.
#print(app1.head(2))
#print(app1.info())

In [ ]:
#application train 데이터와 test 데이터 결합
apps = pd.concat([app_train, app_test])
print(apps.shape)

In [ ]:
# MERGE 사용하여 previous와 apps 조인

prev_app_outer = apps.merge(prev_fe['SK_ID_CURR'], on='SK_ID_CURR', how='outer', indicator=True)

In [ ]:
prev_app_outer['_merge'].value_counts()

In [ ]:
prev_app_outer.head(2)

# 1. EDA
  데이터에서 어떠한 특징을 추출하기 전에 데이터를 잘 살펴봐야함
 데이터의 분포 및 값을 검토함으로써 데이터를 관찰 및 이해를 통해 잠재적인 문제를 발견 가능함
 다양한 각도에서 살펴보는 과정에서 다양한 패턴을 발견할 수 있고, 이를 통해 문제 해결하기 위한 가설을 세울 수 있음

In [ ]:
#TARGET수 비교
#채무이행건이 불이행건에 비해 훨씬 많은 걸 알 수 있다.
app_train['TARGET'].astype(int).plot.hist();

In [ ]:
#TARGET수 비교
#좀 더 보기좋게 시각화 하였음.
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.TARGET.value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of Target')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=app_train,ax=ax[1])
ax[1].set_title('Target count')
plt.show()

In [ ]:
#TARGET값에 따른 데이터 비교

def show_hist_by_target(df, columns):
    cond_0 = (df['TARGET'] == 0)
    cond_1 = (df['TARGET'] == 1)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_0][column], label='0', color='black', ax=ax[0][1])
        sns.distplot(df[cond_1][column], label='1', color='pink', ax=ax[0][1])

In [ ]:
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

show_hist_by_target(app_train, columns)

#### 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'/ 신용평가점수, 'DAYS_BIRTH' 가 다른 데이터들에 비해 TARGET 값에 따라 차이가 존재하는 것을 확인할 수 있음.

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
#kde 커널밀도추적(?)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0', color='pink')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

#나이가 어릴수록 채무불이행이 높은 것을 확인할 수 있다.

In [ ]:
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0', color='pink')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

#EXT_SOURCE_3이 TARGET별 차이가 큰 것을 보아 가장 영향을 많이 줄 것으로 유추됨.

In [ ]:
# TARGET 변수와 EXT_SOURCE와의 상관관계와 EXT_SOURCE 서로간의 상관관계를 살핌

ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap='YlGnBu', vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

#EXT_SOURCE_1과 DAYS_BIRTH의 상관관계가 높은 것을 보아 EXT_SOURCE는 나이와 연관된 것으로 유추됨.

In [ ]:
#오브젝트 데이터의 인덱스 설정
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
#오브젝트 데이터의 인덱스 설정2
object_columns2 = prev_app_outer.dtypes[prev_app_outer.dtypes == 'object'].index.tolist()
object_columns2

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, object_columns)

In [ ]:
#잘 확인되지않는 ORGANIZATION_TYPE만 따로 확인해보았다.
plt.figure(figsize=(5, 15))
sns.barplot(data=app_train, x="TARGET", y="ORGANIZATION_TYPE")

In [ ]:
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(12,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

In [ ]:
# 함수 호출 실패 
# 대출 상태에 따른 Target
#plot_re(prev_app_outer,'AMT_ANNUITY','TARGET')

In [ ]:
#prev데이터의 target별 object 비교

def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(prev_app_outer, object_columns2)

In [ ]:
prev_app_outer.head()

In [ ]:
prev['SK_ID_CURR'].count()

In [ ]:
prev_group = prev.groupby('SK_ID_CURR') # groupby 대상 컬럼명
prev_group.head()

In [ ]:
#id별 과거 대출이력 횟수 시각화
sns.boxplot(prev.groupby('SK_ID_CURR')['SK_ID_CURR'].count())

In [ ]:
sns.jointplot(x="AMT_CREDIT_SUM_LIMIT", y="AMT_CREDIT_SUM_OVERDUE", data=bureau)
plt.show()

# 2. FEATURE ENGINEERING 
데이터에 대한 지식을 활용하여 Target 예측 모델의 정확도를 향상
Target을 분류할 수 있는 피처를 만드는 과정으로 다양한 컬럼(변수) 결합, 상관관계, 피처 변환 등

In [ ]:
# Null 값을 확인하고 전체 데이터에서 비율을 보여주는 함수

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
#prev 데이터에 null data 확인
missing_data(prev).head(5)

In [ ]:
prev.info()

In [ ]:
#prev.groupby('SK_ID_CURR')['SK_ID_CURR'].count()

In [ ]:
#관련성이 높았던 EXT_SOURCE_1,2,3을 활용하여 fe함.

apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
#결측치 제거

apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

In [ ]:
# AMT_CREDIT 비율로 Feature 가공

apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# AMT_INCOME_TOTAL 비율로 Feature 가공

# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공
apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
# 가족수를 고려한 가처분 소득 피처 가공. 
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 Feature 가공

# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공. 
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

In [ ]:
#previous 데이터의 amt_credit의 평균, 최대, 최소값 Feature 가공

#apps['AVG_CREDIT'] = prev_app_outer['AMT_CREDIT'].mean()
#apps['MAX_CREDIT'] = prev_app_outer['AMT_CREDIT'].max()
#apps['MIN_CREDIT'] = prev_app_outer['AMT_CREDIT'].min()

In [ ]:
#id별 과거 대출기록 횟수 
prev_agg = pd.DataFrame()
prev_agg['CNT'] = prev_group['SK_ID_CURR'].count()
prev_agg.head()

In [ ]:
prev1 = prev_agg.reset_index()

In [ ]:
prev1.head()

In [ ]:
#prev 피처 가공 / 대출 신청액 대비 다른 금액 차이 및 비율 생성
apps['PREV_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
apps['PREV_GOODS_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_GOODS_PRICE']
apps['PREV_CREDIT_APPL_RATIO'] = prev['AMT_CREDIT']/prev['AMT_APPLICATION']
apps['PREV_ANNUITY_APPL_RATIO'] = prev['AMT_ANNUITY']/prev['AMT_APPLICATION']
apps['PREV_GOODS_APPL_RATIO'] = prev['AMT_GOODS_PRICE']/prev['AMT_APPLICATION']

In [ ]:
# 과거 대출이력 횟수 생성
apps['CNT'] = prev1['CNT']

In [ ]:
#완성된 FEATURE SET
apps.head(10)

# 모델생성 및 학습

In [ ]:
# 데이터 레이블 인코딩, NULL값은 LightGBM 내부에서 처리하도록 특별한 변경하지 않음

object_columns = apps.dtypes[apps.dtypes=='object'].index.tolist()

for column in object_columns:
    apps[column] = pd.factorize(apps[column])[0] 

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = app_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

학습데이터를 검증데이터로 분리하고 LGBM Classifier로 학습 수행

피처용 데이터와 타겟 데이터 분리

학습용/검증용 데이터 세트 분리

In [ ]:
apps_train = apps[-apps['TARGET'].isnull()]
apps_test = apps[apps['TARGET'].isnull()]
apps.shape, apps_train.shape, apps_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
n_jobs=-1,
n_estimators=1000,
learning_rate = 0.02,
num_leaves=32,
subsample=0.8,
max_depth=12,
silent=-1,
verbose=-1
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
       eval_metric = 'auc', verbose=100, early_stopping_rounds= 50)

### Feature importance 시각화

In [ ]:
from lightgbm import plot_importance
plot_importance(clf, figsize=(16,32))

### 학습된 classfier를 이용하여 테스트 데이터를 예측하고 kaggle로 submit

In [ ]:
preds = clf.predict_proba(apps_test.drop(['SK_ID_CURR', 'TARGET'], axis=1))[:, 1 ]

In [ ]:
app_test['TARGET'] = preds
app_test[['SK_ID_CURR', 'TARGET']].to_csv('apps_baseline_02.csv', index=False)

감사합니다:-)

In [ ]:
#from lightgbm import LGBMClassfier

#clf = LGBMClassfier(
#n_jobs=-1,
#n_estimators = 1000,
#    learning_rate=0.02,
#    num_le